### Comparing models

In [ ]:
# !pip install keras-tcn==2.8.3
# !git clone https://github.com/PatriciaLucas/CNN
# from CNN import Ensemble as es
# from CNN import basic
import Ensemble as es
import basic
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 7
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

from math import sqrt

import warnings
# from google.colab import files
warnings.filterwarnings("ignore")

import gc

In [ ]:
def normalize(df):
    mindf = df.min()
    maxdf = df.max()
    return (df-mindf)/(maxdf-mindf)

def denormalize(norm, _min, _max):
    return [(n * (_max-_min)) + _min for n in norm]

def get_search_dataset_multivariate(dataset, n_var):
    df1 = pd.read_csv(dataset, sep="\s+|;|,") # 7305 registros
    
    ints = df1.select_dtypes(include=['int64','int32','int16']).columns
    df1[ints] = df1[ints].apply(pd.to_numeric, downcast='integer')
    floats = df1.select_dtypes(include=['float']).columns
    df1[floats] = df1[floats].apply(pd.to_numeric, downcast='float')
    
    series = df1.iloc[:,1:n_var+1]
    norm_df = normalize(series)
    size = int(len(norm_df) * 0.80)
    train = series[:size]
    test = series[size:]
    return train, test

def form_data(data, t, n_execucoes, n_previsoes):
    df = pd.DataFrame(data)
    df1 = df.T
    frames = [df1.iloc[:,0], df1.iloc[:,1], df1.iloc[:,2], df1.iloc[:,3], df1.iloc[:,4]]#, df1.iloc[:,5], df1.iloc[:,6], df1.iloc[:,7], df1.iloc[:,8], df1.iloc[:,9], df1.iloc[:,10], df1.iloc[:,11],
        #  df1.iloc[:,12], df1.iloc[:,13], df1.iloc[:,14], df1.iloc[:,15], df1.iloc[:,16], df1.iloc[:,17],df1.iloc[:,18], df1.iloc[:,19], df1.iloc[:,20], df1.iloc[:,21], df1.iloc[:,22], 
        #  df1.iloc[:,23], df1.iloc[:,24], df1.iloc[:,25], df1.iloc[:,26], df1.iloc[:,27], df1.iloc[:,28], df1.iloc[:,29]]
    result = pd.concat(frames)
    r = pd.DataFrame(result) 
    r.insert(1, "Model", True)
    for i in range(n_execucoes * n_previsoes): # n_execucoes * n_previsoes
        r['Model'].iloc[i] = 'CNN'+ t
    return r

def run_model(dataset_file_name, result_file_name, sufix, star, n_var, n_execucoes, n_previsoes):
    train, test = get_search_dataset_multivariate(dataset_file_name, n_var=n_var)
    results = []
    train, test, scaler = es.get_dados(star, train, test)
    X_train, y_train, X_test, y_test = basic.slideWindowMulti(train, test, n_lags=star['lags'], n_var=n_var)
    for i in range(n_execucoes):
        model,_ = basic.modelo_CNN1(X_train, y_train, star)
        rmse, yhat, y_test = basic.predictModelMulti(test, model, n_previsoes=n_previsoes, n_lags=star['lags'], n_var=n_var, scaler=scaler)
        results.append(rmse)

    results = form_data(results, sufix, n_execucoes, n_previsoes)
    results.to_csv(result_file_name,index=True)
    del train
    del test
    del X_train
    del y_train
    del X_test
    del y_test
    del yhat
    del model
    del results
    gc.collect()

In [ ]:
star_CNN1 = {'filters': 1, 'pool': 0, 'pool_size': 3, 'dropout': 0.012594059561340142, 'norm': 1, 'lags': 4, 'num_conv': 1, 'kernel_size': 3, 'rmse': 0.7696852129001718, 'num_param': 449}
# star_CNN2 = {'filters': 1, 'dropout': 0, 'norm': 1, 'lags': 4, 'num_conv': 1, 'kernel_size': 0, 'rmse': 0.7566198577347709, 'num_param': 449}
# star_CNN3 = {'pilhas': 2, 'filters': 1, 'dropout': 0.2, 'norm': 1, 'lags': 48, 'num_conv': 3, 'kernel_size': 2, 'rmse': 0.7530, 'num_param': 68257}

In [ ]:
run_model(dataset_file_name='uni_lat-19.75_lon-44.45.csv', result_file_name='new_u_results_CNN1.csv', sufix='1 (ETo)', star=star_CNN1, n_var=1, n_execucoes=5, n_previsoes=1)

In [ ]:
run_model(dataset_file_name='multi_rs_lat-19.75_lon-44.45.csv', result_file_name='new_m_rs_results_CNN1.csv', sufix='1 (Rs, ETo)', star=star_CNN1, n_var=2, n_execucoes=5, n_previsoes=1)

In [ ]:
run_model(dataset_file_name='multi_u2_lat-19.75_lon-44.45.csv', result_file_name='new_m_u2_results_CNN1.csv', sufix='1 (u2, ETo)', star=star_CNN1, n_var=2, n_execucoes=5, n_previsoes=1)

In [ ]:
run_model(dataset_file_name='multi_rh_lat-19.75_lon-44.45.csv', result_file_name='new_m_rh_results_CNN1.csv', sufix='1 (RH, ETo)', star=star_CNN1, n_var=2, n_execucoes=5, n_previsoes=1)

In [ ]:
run_model(dataset_file_name='multi_tmax_lat-19.75_lon-44.45.csv', result_file_name='new_m_tmax_results_CNN1.csv', sufix='1 (Tmax, ETo)', star=star_CNN1, n_var=2, n_execucoes=5, n_previsoes=1)

In [ ]:
u_CNN1 = pd.read_csv('new_u_results_CNN1.csv',delimiter=',')
u_CNN1 = u_CNN1[u_CNN1['Unnamed: 0'].isin([0,2,6,9])]

m_rs_CNN1 = pd.read_csv('new_m_rs_results_CNN1.csv',delimiter=',')
m_rs_CNN1 = m_rs_CNN1[m_rs_CNN1['Unnamed: 0'].isin([0,2,6,9])]

m_u2_CNN1 = pd.read_csv('new_m_u2_results_CNN1.csv',delimiter=',')
m_u2_CNN1 = m_u2_CNN1[m_u2_CNN1['Unnamed: 0'].isin([0,2,6,9])]

m_rh_CNN1 = pd.read_csv('new_m_rh_results_CNN1.csv',delimiter=',')
m_rh_CNN1 = m_rh_CNN1[m_rh_CNN1['Unnamed: 0'].isin([0,2,6,9])]

m_tmax_CNN1 = pd.read_csv('new_m_tmax_results_CNN1.csv',delimiter=',')
m_tmax_CNN1 = m_tmax_CNN1[m_tmax_CNN1['Unnamed: 0'].isin([0,2,6,9])]

# m_tmin_CNN1 = pd.read_csv('m_tmin_results_CNN1.csv',delimiter=',')
# m_tmin_CNN1 = m_tmin_CNN1[m_tmin_CNN1['Unnamed: 0'].isin([0,2,6,9])]

# m_pr_CNN1 = pd.read_csv('m_pr_results_CNN1.csv',delimiter=',')
# m_pr_CNN1 = m_pr_CNN1[m_pr_CNN1['Unnamed: 0'].isin([0,2,6,9])]

# m_rs_u2_CNN1 = pd.read_csv('m_rs_u2_results_CNN1.csv',delimiter=',')
# m_rs_u2_CNN1 = m_rs_u2_CNN1[m_rs_u2_CNN1['Unnamed: 0'].isin([0,2,6,9])]

# m_u2_tmax_CNN1 = pd.read_csv('m_u2_tmax_results_CNN1.csv',delimiter=',')
# m_u2_tmax_CNN1 = m_u2_tmax_CNN1[m_u2_tmax_CNN1['Unnamed: 0'].isin([0,2,6,9])]

frames = [u_CNN1, m_rs_CNN1, m_u2_CNN1, m_rh_CNN1, m_tmax_CNN1]#, m_tmin_CNN1, m_pr_CNN1, m_rs_u2_CNN1]
result = pd.concat(frames, ignore_index=True)
result['Unnamed: 0'] = result['Unnamed: 0'] + 1

# plt.style.use('seaborn-paper')
sns.set_style("whitegrid")
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[14,6], dpi=100)
g1 = sns.boxplot(x=result.iloc[:,0], y=result.iloc[:,1], hue=result.iloc[:,2], data=result, palette="tab20", linewidth=0.7, saturation=1)
plt.tick_params(labelsize=12)
# plt.xticks([1,3,7,10])
plt.xlabel("Horizontes de previsão", fontsize=14)
plt.ylabel("RMSE", fontsize=14)